# Customer Churn Prediction using ANN

Goal: Build a baseline neural network for churn prediction and improve it iteratively.

## Problem Statement

Customer churn refers to customers leaving a service.
The objective is to predict whether a customer will churn based on their attributes.